In [63]:
import numpy as np
import pandas as pd

In [64]:
train = pd.read_csv("train.csv")

In [65]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [45]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [5]:
temp_df = pd.merge(train.isnull().sum().reset_index() , train.dtypes.reset_index() , on = 'index' , how = 'inner')

In [6]:
temp_df[temp_df['0_x'] > 0]['index'].unique()

array(['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name'], dtype=object)

In [66]:
train_xgb = train.copy(deep = True)

In [8]:
train.dtypes.reset_index()

,index,0
0,PassengerId,object
1,HomePlanet,object
2,CryoSleep,object
3,Cabin,object
4,Destination,object
5,Age,float64
6,VIP,object
7,RoomService,float64
8,FoodCourt,float64
9,ShoppingMall,float64


In [9]:
from sklearn.impute import KNNImputer
float_cols = ['Age' , 'RoomService' , 'FoodCourt' , 'ShoppingMall' , 'Spa', 'VRDeck']
object_cols = ['PassengerId', 'HomePlanet', 'CryoSleep' , 'Cabin', 'Destination', 'VIP' ,	'Name' , 'Transported']
df_float = train[float_cols]
df_object = train[object_cols]
knn_imputer = KNNImputer(n_neighbors=3)
df_float_imputed = pd.DataFrame(knn_imputer.fit_transform(df_float), columns=float_cols)
df_object_imputed = df_object.copy()
for col in object_cols:
    mode_value = df_object[col].mode()[0]  # get most frequent value
    df_object_imputed[col].fillna(mode_value, inplace=True)
train_non_xgb = pd.concat([df_float_imputed, df_object_imputed], axis=1)

C:\Users\Dell\AppData\Local\Temp\ipykernel_20040\1990983576.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_object_imputed[col].fillna(mode_value, inplace=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_20040\1990983576.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_object_imputed[col].fillna(

In [10]:
train_non_xgb[['Cabin' , "CabinA" , "CabinB"]] = train_non_xgb['Cabin'].str.split('/',expand=True)
Y = train_non_xgb['Transported']
X = train_non_xgb.drop(["Transported"] , axis = 1)

X = X.drop(['PassengerId'] , axis = 1)
X = X.drop(["Name"] , axis = 1)
X['CryoSleep'] = X['CryoSleep'].replace({True:1 , False:0})
df_encoded = pd.get_dummies(data = X , columns = ["HomePlanet"])
X.drop(columns=['HomePlanet'], inplace=True)
X = pd.concat([X, df_encoded], axis=1)

df_encoded = pd.get_dummies(data = X , columns = ["Cabin"])
X.drop(columns=['Cabin'], inplace=True)
X = pd.concat([X, df_encoded], axis=1)


df_encoded = pd.get_dummies(data = X , columns = ["CabinB"])
X.drop(columns=['CabinB'], inplace=True)
X = pd.concat([X, df_encoded], axis=1)

df_encoded = pd.get_dummies(data = X , columns = ["Destination"])
X.drop(columns=['Destination'], inplace=True)
X = pd.concat([X, df_encoded], axis=1)

X['VIP'] = X['VIP'].replace({True:1 , False:0})
Y = Y.replace({True:1 , False:0})

C:\Users\Dell\AppData\Local\Temp\ipykernel_20040\2813065888.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X['CryoSleep'] = X['CryoSleep'].replace({True:1 , False:0})
C:\Users\Dell\AppData\Local\Temp\ipykernel_20040\2813065888.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X['VIP'] = X['VIP'].replace({True:1 , False:0})
C:\Users\Dell\AppData\Local\Temp\ipykernel_20040\2813065888.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, expl

In [ ]:
#using logistic regression

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_curve

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
# Train the model

oof_preds = np.zeros(len(X))
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(skf.split(X, Y), 1):
    print(f"Training fold {fold}")

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = Y.iloc[train_idx], Y.iloc[val_idx]

    log_reg = LogisticRegression(max_iter=5000)

    log_reg.fit(
        X_train,
        y_train,
        eval_set=[(X_val, y_val)],
        early_stopping_rounds=50,
        verbose=False
    )

    oof_preds[val_idx] = log_reg.predict_proba(X_val)[:, 1]

precision, recall, thresholds = precision_recall_curve(Y, oof_preds)

f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
best_idx = np.argmax(f1_scores)

BEST_THRESHOLD = thresholds[best_idx]

print("Best threshold:", BEST_THRESHOLD)
print("Best F1:", f1_scores[best_idx])

0.7814836112708453


d:\VS Code\VS\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 5000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=5000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

cv = StratifiedKFold(
    n_splits=10,
    shuffle=True,
    random_state=42
)
rf = RandomForestClassifier(
    random_state=42,
    n_jobs=-1
)
param_dist = {
    "n_estimators": [200, 400, 600],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=50,
    scoring="roc_auc",
    cv=cv,               
    n_jobs=-1,
    random_state=42
)
search.fit(X, Y)


Best Hyperparameters: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_depth': 20, 'bootstrap': False}
0.7929844738355377


In [ ]:
best_clf_decision_tree = search.best_estimator_

In [67]:
train_xgb[['Cabin' , "CabinA" , "CabinB"]] = train_xgb['Cabin'].str.split('/',expand=True)

Y = train_xgb['Transported']
X = train_xgb.drop(["Transported"] , axis = 1)

X = X.drop(['PassengerId'] , axis = 1)
X = X.drop(["Name"] , axis = 1)
Y = Y.replace({True:1 , False:0})

C:\Users\Dell\AppData\Local\Temp\ipykernel_11204\2408022038.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Y = Y.replace({True:1 , False:0})


In [68]:
l = ['Cabin' , 'CabinA' , 'CabinB']
for i in l:
  X[i] = X[i].astype(str).str.replace(" ", "")


In [69]:
X['CabinA'] = X['CabinA'].astype(float)

In [70]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from xgboost.callback import EarlyStopping
cat = ['HomePlanet','Cabin','Destination','CabinB' , 'CryoSleep' , 'VIP']
for i in cat:
    X[i] = X[i].fillna('empty')

In [74]:
for i in cat:
    X[i] = X[i].astype(str).str.replace('nan' , 'empty')

In [79]:
for i in cat:
    X[i] = X[i].astype("category")

In [80]:

cv = StratifiedKFold(
    n_splits=10,
    shuffle=True,
    random_state=42
)
xgb_model = XGBClassifier(
    objective="binary:logistic",
    eval_metric="auc",
    enable_categorical=True,
    tree_method="hist",
    random_state=42,
    n_jobs=-1
)

param_dist = {
    "n_estimators": [300, 500, 800],
    "max_depth": [3, 4, 6, 8],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.7, 0.8, 1.0],
    "colsample_bytree": [0.7, 0.8, 1.0],
    "min_child_weight": [1, 5, 10],
    "gamma": [0, 0.1, 0.3],
    "reg_alpha": [0, 0.1, 1],
    "reg_lambda": [1, 1.5, 2]
}

search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=40,
    scoring="roc_auc",
    cv=cv,                     # <-- Stratified here
    n_jobs=-1,
    random_state=42,
    verbose=1
)

search.fit(X, Y)

best_params = search.best_params_
print(best_params)

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=42)

final_model = XGBClassifier(
    **best_params,
    objective="binary:logistic",
    eval_metric="auc",
    enable_categorical=True,
    tree_method="hist",
    random_state=42,
    n_jobs=-1
)
final_model.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=False
)

Fitting 10 folds for each of 40 candidates, totalling 400 fits
{'subsample': 0.7, 'reg_lambda': 2, 'reg_alpha': 1, 'n_estimators': 800, 'min_child_weight': 1, 'max_depth': 6, 'learning_rate': 0.01, 'gamma': 0.3, 'colsample_bytree': 0.7}


,"objective objective: typing.Union[str, xgboost.sklearn._SklObjWProto, typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]Specify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'binary:logistic'
,"base_score base_score: typing.Union[float, typing.List[float], NoneType]The initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.Optional[typing.List[xgboost.callback.TrainingCallback]]List of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: typing.Optional[float]Subsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: typing.Optional[float]Subsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: typing.Optional[float]Subsample ratio of columns when constructing each tree.,0.7
,"device device: typing.Optional[str].. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: typing.Optional[int].. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,True
,"eval_metric eval_metric: typing.Union[str, typing.List[typing.Union[str, typing.Callable]], typing.Callable, NoneType].. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import load_diabetes fr

In [81]:
import numpy as np
from sklearn.metrics import roc_curve
y_val_proba = final_model.predict_proba(X_val)[:,1]
fpr, tpr, thresholds = roc_curve(y_val, y_val_proba)

j_scores = tpr - fpr
best_idx = np.argmax(j_scores)
best_threshold_roc = thresholds[best_idx]

print("Best threshold (ROC):", best_threshold_roc)

Best threshold (ROC): 0.5761193633079529


In [82]:

q = list(X_train.columns)
features = q
p = []
for i in X_train.columns:
    if X_train[i].dtypes == "category":
        p.append(i)
categorical_cols = p

categorical_map = {}
for i in categorical_cols:
    categorical_map[str(i)] = X_train[i].cat.categories

df_schema = X_train.dtypes.reset_index()
df_schema.columns = ['features' , 'data_type']


q = set(features) - set(categorical_cols)
df_01 = X_train[list(q)].isnull().sum().reset_index()
df_01.columns = ['feature' , 'null_count']

dictionary = {}
for i in categorical_cols:
    dictionary[i] = X_train[X_train[i] == 'empty'].shape[0]

df_09 = pd.DataFrame(list(dictionary.items()), columns=['Key', 'Value'])
df_09.columns = ['feature' , 'null_count']

df_null_count = pd.concat([df_01 , df_09])


dump = {
    'model' : final_model,
    'threshold' : best_threshold_roc, 
    'categorical_cols' : categorical_cols,
    'categorical_map' : categorical_map,
    'features' : features,
    'data_types' : df_schema,
    'missing_value_handelling' : "for categorical features put them empty",
    'null_count' : df_null_count
}

import pickle
with open("best_xgb_model.pkl", "wb") as f:
    pickle.dump(dump, f)


In [32]:
import sklearn
import xgboost
import matplotlib
print("pandas:", pd.__version__)
print("numpy:", np.__version__)
print("sklearn:", sklearn.__version__)
print("xgboost:", xgboost.__version__)
print("matplotlib:", matplotlib.__version__)

pandas: 2.3.3
numpy: 2.3.5
sklearn: 1.8.0
xgboost: 3.1.2
matplotlib: 3.10.8


In [ ]:
#best model is used further for final testing part

In [ ]:
test1 = pd.read_csv("test.csv")

In [18]:
X

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,CabinA,CabinB
0,Europa,0.0,B,TRAPPIST-1e,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0,P
1,Earth,0.0,F,TRAPPIST-1e,24.0,0.0,109.0,9.0,25.0,549.0,44.0,0,S
2,Europa,0.0,A,TRAPPIST-1e,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,0,S
3,Europa,0.0,A,TRAPPIST-1e,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,0,S
4,Earth,0.0,F,TRAPPIST-1e,16.0,0.0,303.0,70.0,151.0,565.0,2.0,1,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,0.0,A,55 Cancri e,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,98,P
8689,Earth,1.0,G,PSO J318.5-22,18.0,0.0,0.0,0.0,0.0,0.0,0.0,1499,S
8690,Earth,0.0,G,TRAPPIST-1e,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,1500,S
8691,Europa,0.0,E,55 Cancri e,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,608,S


In [ ]:
test1.isnull().sum()

,0
PassengerId,0
HomePlanet,87
CryoSleep,93
Cabin,100
Destination,92
Age,91
VIP,93
RoomService,82
FoodCourt,106
ShoppingMall,98


In [ ]:
df = ['HomePlanet' , "CryoSleep" , "Cabin" , "Destination" , "VIP"]
for column in df:
    mode_value = test1[column].mode()[0]
    test1[column].fillna(mode_value, inplace=True)

<ipython-input-31-a4ef7ce253a0>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test1[column].fillna(mode_value, inplace=True)
<ipython-input-31-a4ef7ce253a0>:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test1[column].fillna(mode_value, inplace=True)


In [ ]:
df = ["Age" , "RoomService" , "FoodCourt" , "ShoppingMall" , "Spa" , "VRDeck"]
for column in df:
    mean_value = test1[column].mean()
    test1[column].fillna(mean_value, inplace=True)

<ipython-input-32-1ad840c98d77>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test1[column].fillna(mean_value, inplace=True)


In [ ]:
test1.isnull().sum()

,0
PassengerId,0
HomePlanet,0
CryoSleep,0
Cabin,0
Destination,0
Age,0
VIP,0
RoomService,0
FoodCourt,0
ShoppingMall,0


In [ ]:
test1

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.000000,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.000000,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.000000,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.000000,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.000000,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.000000,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,G/160/P,TRAPPIST-1e,42.000000,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,28.658146,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,TRAPPIST-1e,28.658146,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [ ]:
test1 = test1.drop(["Name"] , axis = 1)
test1['CryoSleep'] = test1['CryoSleep'].replace({True:1 , False:0})
test1[['Cabin' , "CabinA" , "CabinB"]] = test1['Cabin'].str.split('/',expand=True)


<ipython-input-35-f6afe082f913>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test1['CryoSleep'] = test1['CryoSleep'].replace({True:1 , False:0})


In [ ]:
df_encoded = pd.get_dummies(data = test1 , columns = ["HomePlanet"])
test1.drop(columns=['HomePlanet'], inplace=True)
test1 = pd.concat([test1, df_encoded], axis=1)

df_encoded = pd.get_dummies(data = test1 , columns = ["Cabin"])
test1.drop(columns=['Cabin'], inplace=True)
test1 = pd.concat([test1, df_encoded], axis=1)

df_encoded = pd.get_dummies(data = test1 , columns = ["CabinB"])
test1.drop(columns=['CabinB'], inplace=True)
test1 = pd.concat([test1, df_encoded], axis=1)

test1['VIP'] = test1['VIP'].replace({True:1 , False:0})

df_encoded = pd.get_dummies(data = test1, columns = ["Destination"])
test1.drop(columns=['Destination'], inplace=True)
test1 = pd.concat([test1, df_encoded], axis=1)

<ipython-input-38-26a8118bf7f7>:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test1['VIP'] = test1['VIP'].replace({True:1 , False:0})


In [ ]:
test1.drop(columns = ['PassengerId'] , inplace = True)

In [ ]:
Yx = log_reg.predict(test1)
Yx = pd.DataFrame({'Predicted_Label': Yx})
Yx.to_csv('predicted_values_logistic_regression.csv', index=False)

In [ ]:
Yx = best_clf_decision_tree.predict(test1)
Yx = pd.DataFrame({'Predicted_Label': Yx})
Yx.to_csv('predicted_values_logistic_regression.csv', index=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
best_rf_model = RandomForestClassifier(n_estimators = 100, min_samples_split = 10, min_samples_leaf =  2, max_depth =  None, bootstrap =  True)
best_rf_model.fit(X , Y)
Yx = best_rf_model.predict(test1)
Yx = pd.DataFrame({'Predicted_Label': Yx})
Yx.to_csv('predicted_values.csv', index=False)